# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Description**: A brief description of the movie.
  - **Votes**: The number of votes the movie received.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [114]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [120]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def convert_to_minutes(length_str):
    total_minutes = 0
    parts = length_str.split()
    for part in parts:
        if 'h' in part:
            total_minutes += int(part.replace('h', '')) * 60
        elif 'm' in part:
            total_minutes += int(part.replace('m', ''))
    return total_minutes

def scrape_imdb(title_type, user_rating, start_date, end_date):
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    url = f"https://www.imdb.com/search/title/?title_type={title_type}&user_rating={user_rating},&release_date={start_date},{end_date}"
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print("Failed to retrieve data")
        return pd.DataFrame()

    soup = BeautifulSoup(response.content, "html.parser")

    first_box = soup.find_all('h3', class_='ipc-title__text')
    second_box = soup.find_all('span', class_='sc-1e00898e-8 hsHAHC dli-title-metadata-item')
    fourth_box = soup.find_all('div', class_='ipc-html-content-inner-div')
    fifth_box = soup.find_all('div', class_='sc-73c670dc-0')

    numbers = []
    titles = []
    years = []
    movie_lengths = []
    ratings = []
    descriptions = []
    votes = []

    for i, item in enumerate(first_box[:-1]):
        title_text = item.getText().strip()
        number, title = title_text.split('. ', 1)
        numbers.append(number)
        titles.append(title)

        idx = i * 3
        years.append(second_box[idx].text.strip() if len(second_box) > idx else 'N/A')
        length_str = second_box[idx + 1].text.strip() if len(second_box) > idx + 1 else '0m'
        movie_lengths.append(convert_to_minutes(length_str))
        ratings.append(second_box[idx + 2].text.strip() if len(second_box) > idx + 2 else 'N/A')
        
        description = fourth_box[i].text.strip() if len(fourth_box) > i else 'N/A'
        descriptions.append(description)

        vote_text = fifth_box[i].text.strip() if len(fifth_box) > i else 'N/A'
        votes.append(vote_text.split('Votes')[-1].strip())

    df = pd.DataFrame({
        'Movie Nr': numbers,
        'Title': titles,
        'Year': years,
        'Rating': ratings,
        'Runtime (min)': movie_lengths,
        'Description': descriptions,
        'Votes': votes
    })

    df.set_index('Movie Nr', inplace=True)

    return df



In [121]:
title_type = 'feature'
user_rating = '7.5'
start_date = '1990-01-01'
end_date = '1992-12-31'

df = scrape_imdb(title_type, user_rating, start_date, end_date)
df.head()


,Title,Year,Rating,Runtime (min),Description,Votes
Movie Nr,,,,,,
1,O Silêncio dos Inocentes,1991,M/16,118,A young F.B.I. cadet must receive the help of ...,"1,527,583"
2,Mulheres do Sul,1991,M/12,130,A housewife who is unhappy with her life befri...,"81,507"
3,Tudo Bons Rapazes,1990,M/16,145,The story of Henry Hill and his life in the ma...,"1,240,562"
4,Exterminador Implacável 2 - O Dia do Julgamento,1991,M/12,137,"A cyborg, identical to the one who failed to k...","1,161,257"
5,Sozinho em Casa,1990,M/6,103,"An eight-year-old troublemaker, mistakenly lef...","646,664"


## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [117]:
#OUTDATED
